In [ ]:
!cat /proc/cpuinfo

In [ ]:
%%bash
g++ --version
gcc --version

In [ ]:
%%bash
sudo apt install xorg-dev -y

cd ~/content
rm -r ADOP
git clone  --recursive https://github.com/klae01/ADOP.git -j 10
cd ADOP


## Setup Environment
## Install Pytorch from Source

In [ ]:
%%bash
export USE_ROCM=0
cd ~/content
cd ADOP
./create_environment.sh
./install_pytorch.sh

## Build ADOP


In [ ]:
%%bash
cd ~/content/ADOP
conda activate adop

# Set this to either g++-7 or 9
export CC=gcc-9
export CXX=g++-9
export CUDAHOSTCXX=g++-9

mkdir build
cd build
export CONDA=${CONDA_PREFIX:-"$(dirname $(which conda))/../"}
PYVERSION=(python3 -c 'import sys; print(f"{sys.version_info.major}.{sys.version_info.minor}")')
cmake -DCMAKE_PREFIX_PATH="${CONDA}/lib/python${PYVERSION}/site-packages/torch/;${CONDA}" ..
make -j

In [ ]:
%%bash
sudo ls -l /dev/nvidia*
sudo usermod -a -G vglusers $USER

In [ ]:
%%bash
DATASET_PATH=~/content/CASE/DATA
USE_VIDEO=1
cd $DATASET_PATH
rm -r dense
rm -r sparse
rm database.db*

if [ USE_VIDEO ]; then
    rm -r images
    mkdir images
    fps=4
    ffmpeg -i *.mp4 -qscale:v 1 -qmin 1 -vf "fps=$fps" images/%04d.jpg
fi

colmap feature_extractor \
   --database_path $DATASET_PATH/database.db \
   --image_path $DATASET_PATH/images

if [ USE_VIDEO ]; then
    colmap sequential_matcher \
        --database_path $DATASET_PATH/database.db
else
    colmap exhaustive_matcher \
        --database_path $DATASET_PATH/database.db
fi

mkdir $DATASET_PATH/sparse

colmap mapper \
    --database_path $DATASET_PATH/database.db \
    --image_path $DATASET_PATH/images \
    --output_path $DATASET_PATH/sparse

mkdir $DATASET_PATH/dense

colmap image_undistorter \
    --image_path $DATASET_PATH/images \
    --input_path $DATASET_PATH/sparse/0 \
    --output_path $DATASET_PATH/dense \
    --output_type COLMAP \
    --max_image_size 2000

# using gpu 
colmap patch_match_stereo \
    --workspace_path $DATASET_PATH/dense \
    --workspace_format COLMAP \
    --PatchMatchStereo.geom_consistency true

colmap stereo_fusion \
    --workspace_path $DATASET_PATH/dense \
    --workspace_format COLMAP \
    --input_type geometric \
    --output_path $DATASET_PATH/dense/fused.ply

export SCENE_BASE=$DATASET_PATH
mkdir $SCENE_BASE/adop_input
~/content/ADOP/build/bin/colmap2adop --sparse_dir $SCENE_BASE/sparse/0/ \
    --image_dir $SCENE_BASE/images/ \
    --point_cloud_file $SCENE_BASE/dense/fused.ply \
    --output_path $SCENE_BASE/adop_input \
    --scale_intrinsics 1 --render_scale 1 


In [1]:
%%bash
# sudo ln -sfn /usr/local/cuda/lib64/libnvrtc-builtins.so.11.6 /usr/local/cuda/lib64/libnvrtc-builtins.so.11.2 
export LD_LIBRARY_PATH="~/.conda/envs/adop/lib/:${LD_LIBRARY_PATH}" 
cd  ~/content/ADOP/
sudo build/bin/adop_train --config $DATASET_PATH/train_config.ini

build/bin/adop_train: error while loading shared libraries: libnvToolsExt.so.1: cannot open shared object file: No such file or directory


CalledProcessError: Command 'b'# sudo ln -sfn /usr/local/cuda/lib64/libnvrtc-builtins.so.11.6 /usr/local/cuda/lib64/libnvrtc-builtins.so.11.2 \ncd  ~/content/ADOP/\nsudo build/bin/adop_train --config $DATASET_PATH/train_config.ini\n'' returned non-zero exit status 127.

In [ ]:
# unzip -l scenes.zip
# reorganize path for train
#   

In [ ]:
%%bash
cd /content
cd ADOP

./build/bin/adop_train --config configs/train_boat.ini

In [ ]:
%%bash
cd /content

In [ ]:
%%bash
cd /content